# Lab06: Modelling Plant Gas-Exchange Process

<div class="alert alert-warning">
If figures/tables are not rendered correctly, make sure your notebook is `trusted` by choosing `Trust Notebook` command found in `View` - `Activate Command Palette` menu. Typing in the first letters (i.e. "tr") should give you the command.
</div>

In [ ]:
using Cropbox

We will use interactive plotting feature in this notebook. For this, we have to initialize [WebIO.jl](https://github.com/JuliaGizmos/WebIO.jl) manually when using [prebuilt system images](https://github.com/JuliaLang/PackageCompiler.jl) as we do with our JupyterHub environment.

In [ ]:
Cropbox.Interact.WebIO.setup(:ijulia)

Bisection method used for optimization by default prints out warning messages when it can't find a solution within a given range, for example, when we try running simulation under an extreme condition (*i.e.* some zero values for parameters). Let's make it quiet.

In [ ]:
using Logging
Logging.disable_logging(Logging.Warn)

## The Leaf Gas-Exchange Model

The leaf gas-exchange model was originally derived from [Kim and Lieth (2003)](https://doi.org/10.1093/aob/mcg080) with an addition of C4 photosynthesis and stress responses based on [MAIZSIM](https://github.com/ARS-CSGCL-DT/MAIZSIM). The model was reimplemented in the Cropbox framework to build a garlic model [Garlic.jl](https://github.com/cropbox/Garlic.jl). 

A standalone package for couple leaf gas-exchange model is available as [LeafGasExchange.jl](https://github.com/cropbox/LeafGasExchange.jl). Description of the standalone gas-exchange model is available from [Yun, Timlin, and Kim (2020)](https://doi.org/10.3390/plants9101358) with applications focusing on C4 leaves. We will use this standalong package to study leaf gas-exchange behavior without going into the model details. If yor're intersted in the internals of how the model was put together, pleaes refer to the optional notebook on its implementation placed in the same folder.

In [ ]:
using LeafGasExchange

#### $\mathrm{CO_2}$ diffusion equation: The supply function

In order for $\mathrm{CO_2}$ assimilation to take place at the cholorplast, the biochemical demand for $\mathrm{CO_2}$ to run the photosynthetic machinery (the Calvin cycle) must be met by the supply of $\mathrm{CO_2}$ (substrate) from the atmosphere. 

$$
{r_t} = {r_b} + {r_s} + {r_m}
$$

To reach the site of caboxylation at the choloroplast, $\mathrm{CO_2}$ molecules in the air encouter multiple resistences ($r$) in their diffusional pathway to travel through the leaf boundary layer ($r_b$), stomatal pores ($r_s$), interceullar air-spaces ($r_{ias}$), and mesophyll cell walls ($r_{cw}$)  inside the leaf before dissolving into the liquid phase ($r_{liq}$) and enter the Calvin cycle. These resistences are connected in series along the $\mathrm{CO_2}$ diffusional pathway. Hence, the sum of all resistences determine the total resistence ($r_t$). The last three terms in the pathway (i.e., $r_{ias}, r_{cw}, r_{liq}$) matter only after $\mathrm{CO_2}$ enters into the inside of a leaf. Lumping them together, these terms are referred as the mesophyll resistence ($r_m$) or internal resistence ($r_i$).  

$$
\frac{1}{g_t} = \frac{1}{g_b} + \frac{1}{g_s} + \frac{1}{g_m} \approx \frac{1}{g_b} + \frac{1}{g_s}
$$

While it may make intuititve sense to use resistence terms in physics, we use conductance terms which are  reciprocal of the resistence terms in ecophysiology as they better represent biological and physiological agencies that could facilitate or inhibit the diffusion of $\mathrm{CO_2}$ and $\mathrm{H_2O}$ in the pathway. That is, we use $g_b$ for boudary later conductance, $g_s$ for stomatal conductance, and $g_m$ for mesophyll conductance that lumps all internal conductances (i.e., intercellular spaces, cell wall, and liquid phase) together. $g_t$ is the total conductance that combines all counductances. Note that all conductance terms are for water vapor ($\mathrm{H_2O}$), not for $\mathrm{CO_2}$.

Here, an important assumption we make is that the mesophyll conductance is infinite (that is, no internal resistence between sub-stomatal cavity and the chloroplast once $\mathrm{CO_2}$ enters a stoma. This means we also assume $C_i$ $\simeq$ $C_c$. The total conductance for $\mathrm{H_2O}$ is divided by 1.6 here to account for the slower diffusion of $\mathrm{CO_2}$ due to heavier molecular weight. If $g_b$ is sufficiently large, we may also assume: $g_t$ $\approx$ $g_s$. Putting these relationships and assumptions together based on the Fick's Law, we get the $\mathrm{CO_2}$ supply function as: 

$$
A = \frac{g_{t}}{1.6} (C_a - C_i) \simeq \frac{g_{s}}{1.6} (C_a - C_i)
$$

#### Biochemical model of C3 photosynthesis (FvCB): The demand function

$$
A = \min \{ A_c, A_j, A_p \}
$$

$$
A_c = V_{c\!\max} \frac{C_i - \Gamma^*}{C_i + K_c \cdot (1 + {O_i}/{K_o})} - R_d
$$

$$
A_j = \frac{J \cdot (C_i - \Gamma^*)}{4 (C_i + 2 \Gamma^*)} - R_d
$$

$$
A_p = 3T_p - R_d
$$

#### Stomatal conductance model (Ball-Berry): The coupler

$$
g_s = g_0 + g_1 \frac{h_s A}{C_s}
$$

#### Model Parameters and Variables ####
| Symbol | Units | Description |
|:-------|:------|:------------|
|| **The FvCB C3 photosynthesis model** ||
| $A$ | $\mathrm{\mu mol_{CO_2}}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | Net $\mathrm{CO_2}$ assimilation rate |
| $A_c$ | $\mathrm{\mu mol_{CO_2}}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | Rubisco-limited $\mathrm{CO_2}$ assimilation rate |
| $A_j$ | $\mathrm{\mu mol_{CO_2}}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | Electron transport-limited $\mathrm{CO_2}$ assimilation rate |
| $A_p$ | $\mathrm{\mu mol_{CO_2}}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | Triose phosphate utilization-limited $\mathrm{CO_2}$ assimilation rate |
| $C_a$ | $\mathrm{\mu bar}$ | Atmospheric $\mathrm{CO_2}$ partial pressure |
| $C_i$ | $\mathrm{\mu bar}$ | Intercellular $\mathrm{CO_2}$ partial pressure |
| $\Gamma^*$ | $\mathrm{\mu bar}$ | $\mathrm{CO_2}$ compensation point when $A$ = 0 and $R_d$ = 0|
| $J$ | $$\mathrm{\mu mol_{electron}}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$$ | Electron transport rate |
| $J_{\max}$ | $\mathrm{\mu mol_{electron}}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | Maximum rate of electron transport |
| $K_c$ | $\mathrm{\mu bar}$ | Michaelis-Menton constant of Rubisco for $\mathrm{CO_2}$ |
| $K_o$ | $\mathrm{mbar}$ | Michaelis-Menton constant of Rubisco for $\mathrm{O_2}$ |
| $R_d$ | $\mathrm{\mu mol_{CO_2}}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | Mitochondrial respiration rate |
| $O_i$ | $\mathrm{mbar}$ | Intercellular $\mathrm{O_2}$ partial pressure |
| $T_p$ | $\mathrm{\mu mol_{CO_2}}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | Maximum rate of triose phosphate utilization |
| $V_{c\!\max}$ | $\mathrm{\mu mol_{CO_2}}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | Maximum rate of Rubisco carboxylation |
||**The Ball-Berry Stomatal Counductance Model** ||
| $g_s$ | $\mathrm{mol_{H_2O}}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | Stomatal conductance for water vapor |
| $g_t$ | $\mathrm{mol_{H_2O}}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | Total conductance for water vapor in the diffusion pathway |
| $g_0$ | $\mathrm{mol_{H_2O}}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | Minimal stomatal conductance $\mathrm{CO_2}$ due to cuticular and residual water loss (also known as $b$ parameter |
| $g_1$ |  | Empirical parameter related to stomatal sensitivity to the marginal water cost of carbon gain (also know as $m$ parameter)  |
| $h_s$ | $ - $ | Relative humidity at the leaf surface as fraction |



`LeafGasExchange.jl` package implements gas-exchange model for C3 and C4 leaves. Two types of stomatal conductance models, Ball-Berry (BB) and Medlyn (MD) are supported. For example, `ModelC3BB` indicates leaf-level gas-exchange model for C3 leaf coupled with Ball-Berry model.

In [ ]:
Cropbox.hierarchy(ModelC3BB; skipcontext = true)

In [ ]:
Cropbox.dependency(ModelC3BB)

## Exercises

#### Ex 6.2.

Use PGES or Cropbox to examine how photosynthesis, stomatal conductance, and transpiration at the leaf level will respond to light (PFD; photosynthetic photon flux density), $\mathrm{CO_2}$ in the air, air temperature (Air T), relative humidity (RH), and soil or leaf water potential (SWP or WP_leaf) with default leaf traits for rose and maize. Evaluate and discuss the results. How are they similar or different? Do model predictions make sense to you? Discuss why or why not.

To answer these questions, we first need to check the deafult parameter values for C3 photosynthesis and Ball-Berry stomatal conductatance models implemented in LeafGasExchange package, and make a new instance for species of interest. We will focus on rose, a C3 plant. The same approach method can apply for maize which is a C4 plant.

In [ ]:
c1 = parameters(LeafGasExchange.ModelC3BB);

From [Kim and Lieth (2003)](https://doi.org/10.1093/aob/mcg080), we see that some of the FcVB and Ball-Berry model parameter values for rose leaves are different than the default values. We will create a new parameter configuration for a rose leaf and update their parameter values as described in [Kim and Lieth (2003)](https://doi.org/10.1093/aob/mcg080). The current version of LeafGasExchange package rquires optimal temperature for J (Toj). Toj can be obtained from the function below based on Sj, Eaj (or Haj), and Hj (or Hdj).

In [ ]:
# Create a gas-exchange configuration for rose leaf
ge_rose = @config (
    :C3 => (
        Vcm25 = 102.4, Jm25 = 162.0, Tp25 = 11.55, Rd25 = 1.260,
        Kc25 = 404, Ko25 = 248, Eac = 59.4, Eao = 36.0, Ear = 66.4, EaVc = 45.5,
        Eaj = 43.3, Hj = 219.4, Sj = 704.2, Γ25 = 36.9,
    ),
    :StomataBallBerry => (g0 = 0.03, g1 = 10.0), 
     # we will use a generic g0 and g1 values for C3 leaves instead of the values calibrated for rose leaves (0.096, 10.055) 
    :Weather => (
        PFD = 1500,
        CO2 = 400,
        RH = 60,
        T_air = 25,
        wind = 2.0,
    )
);

In [ ]:
#list all parameters for rose
@config c1 + ge_rose; 

In [ ]:
# Generate CO2 response curves (aka A-Ci curve) for all three limiting steps
visualize(LeafGasExchange.ModelC3BB, :CO2, [:Ac, :Aj, :Ap];
    config = ge_rose,
    xstep = :Weather => :CO2 => 10:10:1500,
    ylim = (0,40),
    kind = :line,
)

In [ ]:
# Generate light response curves (aka A-Q curve) for all three limiting steps 
visualize(LeafGasExchange.ModelC3BB, :PFD, [:Ac, :Aj, :Ap];
    config = ge_rose,
    xstep = :Weather => :PFD => 10:10:2000,
    ylim = (0,40),
    kind = :line,
)

In [ ]:
# Generate temperature response curves of A and its three limiting steps 
visualize(LeafGasExchange.ModelC3BB, :T, [:Ac, :Aj, :Ap];
    config = ge_rose,
    xstep = :Weather => :T_air => 0:1:45,
    ylim = (0,30),
    kind = :line,
)

In [ ]:
# Generate relative humidity response curves of A and its three limiting steps 
visualize(LeafGasExchange.ModelC3BB, :RH, [:Ac, :Aj, :Ap];
    config = ge_rose,
    xstep = :Weather => :RH => 20:5:100,
    ylim = (0,40),
    kind = :line,
)

In [ ]:
# Generate leaf water potential response curves of A and its three limiting steps
visualize(LeafGasExchange.ModelC3BB, :WP_leaf, [:Ac, :Aj, :Ap];
    config = ge_rose,
    xstep = :StomataTuzet => :WP_leaf => -5:0.02:0,
    ylim = (0,30),
    kind = :line,
)

#### Ex 6.3.

Based on the coupled models of C3 and C4 leaves, generate $A/C_i$ and light response ($A/Q$) curves for C3 (rose) and C4 (maize) leaves, and identify regions limited by each of the three steps (*i.e.*, $A_c$, $A_j$, or $A_p$). Assume that all other conditions are optimal for photosynthesis except the variable of interest (*i.e.*, $\mathrm{CO_2}$ and PFD).

**Since we did the C3 part of this exercise for a rose leaf above, we will just do the remaining part for a C4 leaf.**

In [ ]:
# We could take specific modules out to public for referring to them directly as variable. Otherwise, we refer to them as symbol with ":"
using LeafGasExchange: Weather

In [ ]:
ge_base = @config (
    Weather => (;
        PFD = 2000,
        CO2 = 400,
        RH = 60,
        T_air = 30,
        wind = 2.0,
    ),
)

In [ ]:
co2_xstep = Weather => :CO2   => 10:10:1500;
pfd_xstep = Weather => :PFD   => 0:20:2000;
ta_xstep  = Weather => :T_air => 0:1:45;
rh_xstep  = Weather => :RH    => 0:1:100;

### C4

#### CO2 response

In [ ]:
visualize(ModelC4BB, :CO2, [:Ac, :Aj, :A_net];
    config = ge_base,
    xstep = co2_xstep,
    ylim = (0,70),
    kind = :line,
)

#### Light response

In [ ]:
visualize(ModelC4BB, :PFD, [:Ac, :Aj, :A_net];
    config = ge_base,
    xstep = pfd_xstep,
    ylim = (0,70),
    kind = :line,
)

#### Temperature response

In [ ]:
visualize(ModelC4BB, :T_air, [:Ac, :Aj, :A_net];
    config = ge_base,
    xstep = ta_xstep,
    ylim = (0,70),
    kind = :line,
)

#### Transition smoothing factor ($\beta$)

In [ ]:
let S = ModelC4BB, x = :T_air, y = :A_net, c = ge_base, xstep = ta_xstep, ylim = (0,60), kind = :line
    p = visualize(S, x, y; xstep, ylim, kind, name = "β = 1", config = (c, :0 => :β => 1))
    p = visualize!(p, S, x, y; xstep, kind, name = "β = 0.99", config = (c, :0 => :β => 0.99))
    p = visualize!(p, S, x, y; xstep, kind, name = "β = 0.9", config = (c, :0 => :β => 0.9))
    p = visualize!(p, S, x, y; xstep, kind, name = "β = 0.8", config = (c, :0 => :β => 0.8))
end

In [ ]:
manipulate(ModelC4BB, :T_air, :A_net;
    parameters = :0 => :β => 0.01:0.01:1,
    config = ge_base,
    xstep = ta_xstep,
    ylim = (0,60),
    kind = :line,
)

#### Ex 6.4.

Explore how a rose (C3) and maize (C4) leaves would respond to a range of temperatures under the atmospheric CO2 concentrations of **250 ppm (sub-ambient), 400 ppm (ambient), and 1000 ppm (elevated)**. Assume other environmental conditions are constant and favorable under saturating light level (PFD = 2,000 $\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$).

In [ ]:
co2_group = Weather => :CO2   => [1000, 400, 250];
pfd_group = Weather => :PFD   => 1800:-400:600;
ta_group  = Weather => :T_air => 40:-5:10;
rh_group  = Weather => :RH    => 80:-20:20;

#### Ex 6.4.a.

Generate figures to compare C3 and C4 temperature response curves at the three CO2.

In [ ]:
visualize(ModelC3BB, :T, :A_net;
    config = ge_rose,
    xstep = ta_xstep,
    group = co2_group,
    ylim = (0,40),
    kind = :line,
)

In [ ]:
visualize(ModelC4BB, :T, :A_net;
    config = ge_base,
    xstep = ta_xstep,
    group = co2_group,
    ylim = (0,60),
    kind = :line,
)

#### Ex 6.4.b.

Identify the **air temperature and CO2** conditions where photorespiration becomes substantial in both species, and discuss how you identified those conditions.

**Quick and easy way to do this is to examine the temperature response curves above to identify where the gaps are between lines**

**[Extra] Next cell shows how photorespiration can be calculated**

In [ ]:
#Rubisco kinetics for carboxylation and oxygenation, See von Caemmerer (2000), Biochemical models of leaf photosynthesis
@system Rubisco(ModelC3BB, Controller) begin
    Vomax(Vcmax): rubisco_oxygenation_capacity => begin 0.25Vcmax  end ~ track(u"μmol/m^2/s")
    s_co(Kc,Ko,Vcmax, Vomax): rubisco_specificity => begin (Vcmax*Ko)/(Kc*Vomax) end ~ track(u"mmol/μmol")
    phi(s_co, Om,Ci): ratio_oxygenation_to_carboxylation => begin (1/s_co)*(Om/Ci) end ~ track()
    Photorespiration(Γ,Ci,A_net,Rd): photorespiration_rate => begin (Γ/(Ci-Γ))*(A_net+Rd) end ~track(u"μmol/m^2/s")
end

In [ ]:
visualize(Rubisco, :T, :Photorespiration;
    config = ge_rose,
    xstep = ta_xstep,
    group = co2_group,
    ylim = (0,10),
    kind = :line,
)

In [ ]:
# examine how C4 photosynthesis response to CO2 at different temperatures, and vice versa
manipulate(ModelC4BB, :CO2, [:Ac, :Aj, :A_net];
    config = ge_base,
    parameters = ta_xstep,
    xstep = co2_xstep,
    xlim = (0,500),
    ylim = (0,60),
    kind = :line,
)

#### Ex 6.4.c.

Do the same comparisons with water-stressed C3 rose leaf at -1.5 MPa of soil water potential (or -3.0 MPa of leaf water potential). Compare their responses to fully turgid leaf with water potential close to 0 MPa.

In [ ]:
manipulate(ModelC3BB, :T, :A_net;
    parameters = :StomataTuzet => :WP_leaf => -3:0.1:0,
    config = ge_rose,
    xstep = ta_xstep,
    group = co2_group,
    xlim = (0,50),
    ylim = (0,50),
    kind = :line,
)

In [ ]:
manipulate(ModelC4BB, :T, :A_net;
    parameters = :StomataTuzet => :WP_leaf => -3:0.1:0,
    config = ge_base,
    xstep = ta_xstep,
    group = co2_group,
    xlim = (0,50),
    ylim = (0,60),
    kind = :line,
)

## The A-Ci Response Curve

In [ ]:
# Examine the relationship between supply and demand functions of photosynthesis
import Cropbox.Gadfly as G
visualize_ACi(S, targets = [:A_net], Ca = nothing;
    config = (),
    xstep = :Weather => :CO2 => 50:50:1500,
    kw...
) = begin
    c = @config(config)
    R = simulate(S; target = [:Ci, :Ca, :CO2, :gs, targets...], configs = @config(c + !xstep))
    p = plot(R, :Ci, targets; kind = :line, kw...)
    for r in eachrow(deunitfy.(R))
        x1, y1 = r.Ci, r.A_net
        x2, y2 = r.Ca, 0
        m = (y2 - y1) / (x2 - x1)
        f = x -> m * (x - x1) + y1
        if isequal(r.CO2, Cropbox.option(c, :Plot, :CO2))
            theme = G.Theme(default_color = "gray", line_style = [:solid])
            info = "An = $(round(r.A_net, digits=1)) μmol/m^2/s\ngs = $(round(r.gs, digits=3)) mol/m^2/s/bar"
            guide = G.Guide.annotation(
                G.compose(
                    G.context(),
                    G.Compose.text((x1+x2)/2 + 20, (y1+y2)/2, info),
                    G.fontsize(theme.minor_label_font_size),
                )
            )
            G.push!(p[], guide)
        else
            theme = G.Theme(default_color = "lightgray", line_style = [:dot])
        end
        l = G.layer(f, x1, x2, theme)
        G.push!(p[], l)
    end
    p
end

In [ ]:
manipulate(config = ge_rose, parameters = (
    :StomataTuzet => :WP_leaf => -3:0.1:0,
    :Weather      => :PFD     => 0:100:2000,
    :Weather      => :T_air   => 0:1:50,
    :Plot         => :CO2     => 50:50:1000,
)) do c
    visualize_ACi(LeafGasExchange.ModelC3BB, [:A_net, :Ac, :Aj, :Ap];
        config = c,
        xstep = :Weather => :CO2 => 50:50:1500,
        xlim = (0, 1000),
        ylim = (0, 40),
    )
end

## Canopy Gas-Exchange Responses

For the following exercises, we're going to use gas-exchange model scaled to canopy level. Canopy-level models are named with suffix `C`. For example, `ModelC3BBC` is canopy-level C3 gas-exchange model coupled with Ball-Berry stomatal conductance model. We will be working with a rose canopy.

In [ ]:
# We could take specific modules out to public for referring to them directly as variable. Otherwise, we refer to them as symbol with ":"
using LeafGasExchange: C3, Sun, Canopy, Radiation, LeafAngle

In [ ]:
ge_canopy = @config(
    ge_rose,
    C3 => (;
        Vcm25 = 102.4,
        Jm25 = 162.0,
    ),
    Sun => (;
        day = 100,
        hour = 12,
    ),
    Canopy => (;
        LAI = 5,
    ),
    Radiation => (;
        leaf_angle_factor = 3,
        leaf_angle = LeafGasExchange.spherical,
    )
)

In [ ]:
instances(LeafAngle)

In [ ]:
manipulate(ModelC3BBC, "weather.CO2", [:A_net, "sunlit_gasexchange.A_net", "shaded_gasexchange.A_net"];
    config = ge_canopy,
    parameters = (
        Canopy => :LAI => 0:0.1:20,
        Radiation => :leaf_angle => LeafAngle,
    ),
    xstep = co2_xstep,
    ylim = (0,70),
    kind = :line,
    xlab = "CO2",
    ylab = "A_net",
    names = ["Total", "Sunlit", "Shaded"],
)